<h2>Queremos fazer uma análise de quais partidos,
mais especificamente governos de UFs que foram mais permissivos 
em relação à degradação do meio ambiente.
Para isso, precisamos de dados do meio ambiente e de eleições.

<h3>Instalando base dos dados

In [1]:
!pip install basedosdados

You should consider upgrading via the 'd:\python\python.exe -m pip install --upgrade pip' command.


Importando ferramentas

In [2]:
!pip install plotly

You should consider upgrading via the 'd:\python\python.exe -m pip install --upgrade pip' command.


In [3]:
import basedosdados as bd
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import plotly.offline as py
import plotly.graph_objs as go

<h3>Lendo tabela de informação de candidatos eleitos

<h4> Para pegar as informações relevantes, selecionamos só os governantes de UFs que têm dados sobre sua cobertura de mata amazônia na basedosdados 
     <h4>A consulta em si está na pasta de consultas SQL com o nome 'GovernoEstado.sql'
         <h4> Aqui importamos o csv gerado pela consulta descrita acima

In [4]:
csvGov= pd.read_csv('ConsultasEmCsv/governosestados.csv')

In [5]:
csvGov.head(15)

,anos,sigla_uf,sigla_partido
0,1994,AC,PPR
1,1998,AC,PT
2,2002,AC,PT
3,2006,AC,PT
4,2010,AC,PT
5,2014,AC,PT
6,2018,AC,PP
7,1994,AM,PPR
8,1998,AM,PFL
9,2002,AM,PPS


<h3>Lendo tabela de informação de cobertura do bioma amazônia por estado e ano

<h4> Pegamos as informações da área do bioma Amazônia relacionada ao estado e ao ano em que foram registradas
     <h4>A consulta em si está na pasta de consultas SQL com o nome 'AreaTotAmazonia.sql'
         <h4> Aqui importamos o csv gerado pela consulta descrita acima

In [6]:
csvCob= pd.read_csv('ConsultasEmCsv/coberturaanoestado.csv')

In [7]:
csvCob

,ano,estado_abrev,areatotal
0,1985,AC,158629.949213
1,1986,AC,158481.673831
2,1987,AC,158058.639895
3,1988,AC,157346.219951
4,1989,AC,156778.336434
...,...,...,...
310,2015,TO,6951.741695
311,2016,TO,6592.706262
312,2017,TO,6853.481358
313,2018,TO,6645.391944


<h1> Queremos saber quais partidos foram mais permissivos em relação a perda do bioma da amazônia

<h3> Relacionando as duas tabelas para obter o dataset o qual utilizaremos para operações e análises no pandas, com o objetivo de responder a pergunta acima:
   

<h4> Com as duas tabelas acima, fazemos um inner join para relacionar as mudanças dos biomas com o partido governante


<p> Fazemos isso utilizando funções do pandas

In [209]:
csvCob = csvCob.rename(columns = {'estado_abrev' : 'sigla_uf', 'anos' : 'ano' })
csvGov = csvGov.rename(columns = {'anos' : 'ano'})

#fazemos o join entre os dataframes de interesse
lig = csvCob.merge(csvGov, how = 'inner', on= ['ano', 'sigla_uf'])
lig.head()

,ano,sigla_uf,areatotal,sigla_partido
0,1994,AC,154742.431297,PPR
1,1998,AC,152234.087485,PT
2,2002,AC,148017.291313,PT
3,2006,AC,147719.725350,PT
4,2010,AC,146517.504490,PT


<h3> Adicionando coluna na qual queremos armazenar a porcentagem de cobertura de amazônia perdida por mandato de governo

In [145]:
#Usando o pandas, adicionando uma coluna inicialmente vazia
lig["delta"] = None
lig.head(15)

,ano,sigla_uf,areatotal,sigla_partido,delta
0,1994,AC,1.547424e+05,PPR,None
1,1998,AC,1.522341e+05,PT,None
2,2002,AC,1.480173e+05,PT,None
3,2006,AC,1.477197e+05,PT,None
4,2010,AC,1.465175e+05,PT,None
5,2014,AC,1.456493e+05,PT,None
6,2018,AC,1.441178e+05,PP,None
7,1994,AM,1.505151e+06,PPR,None
8,1998,AM,1.503607e+06,PFL,None
9,2002,AM,1.503723e+06,PPS,None


<h3>Função para armazenar a perda de cobertura do bioma por mandato
    <h4> Nessa função, percorremos as linhas do database e pegamos a informação da área antes e após o mandato, para armazenar a porcentagem de quanto mudou na coluna delta

In [158]:
lig["delta"] = None
lig["perdatot"]= None
for index in lig.index:
    line = lig.iloc[index]
    ano = line.ano
    est = line.sigla_uf
    areaantiga = line.areatotal
    areaatual = lig[(lig.sigla_uf==est) & (lig.ano == ano + 4)].areatotal
    if len(areaatual) == 0:
        areaatual= None
    else:
        areaatual= areaatual.iloc[0]
        lig.loc[index,'delta'] = (-1)*((areaatual - areaantiga)/ areaantiga)*100
        lig.loc[index, 'perdatot'] = areaantiga - areaatual

#variável 'deltadb' agora armazena os deltas de áreas
deltadb = lig
#removemos linhas em que nao temos informações do desmatamento
deltadb = deltadb.dropna()

In [156]:
deltadb.head(15)

,ano,sigla_uf,areatotal,sigla_partido,delta,perdatot
0,1994,AC,1.547424e+05,PPR,1.62098,2508.343812
1,1998,AC,1.522341e+05,PT,2.769942,4216.796173
2,2002,AC,1.480173e+05,PT,0.201035,297.565963
3,2006,AC,1.477197e+05,PT,0.813853,1202.22086
4,2010,AC,1.465175e+05,PT,0.592542,868.178236
5,2014,AC,1.456493e+05,PT,1.051536,1531.555535
7,1994,AM,1.505151e+06,PPR,0.102529,1543.20929
8,1998,AM,1.503607e+06,PFL,-0.007702,-115.813629
9,2002,AM,1.503723e+06,PPS,0.401206,6033.033481
10,2006,AM,1.497690e+06,PMDB,0.112286,1681.694831


<h4> Com a porcentagem da área perdida relacionada com o partido que estava no mandato, podemos fazer cálculos para comparar quem foi mais permissivo em relação à perda da biodiversidade

In [149]:
#Vamos usar as duas colunas, a que indica o partido que governou, e o saldo do mandato em relação a área do bioma
ptbioma= deltadb[['sigla_partido','delta'] ]
ptbioma.head(15)

,sigla_partido,delta
0,PPR,1.62098
1,PT,2.769942
2,PT,0.201035
3,PT,0.813853
4,PT,0.592542
5,PT,1.051536
7,PPR,0.102529
8,PFL,-0.007702
9,PPS,0.401206
10,PMDB,0.112286


<h3> Somamos porcentagens por cada partido e fazemos uma média em relação a quantos mandatos o partido teve, para poder comparar qual foi o mais permissivo

<h4>Somando porcentagens por partido:

In [159]:
a= ptbioma.groupby('sigla_partido').delta.sum()

<h4>Contando instâncias de partidos para fazer a média

In [160]:
b = ptbioma.groupby('sigla_partido').sigla_partido.count()

<b>Pegando series dos partidos

In [161]:
pts = pd.unique(ptbioma ['sigla_partido'])
pts = pd.Series(pts)
pts

0         PPR
1          PT
2         PFL
3         PPS
4        PMDB
5         PMN
6        PROS
7         PSB
8         PDT
9     PC do B
10       PSDB
11        PTB
12        PPB
13        PSL
14         PP
dtype: object

<b> Pegando a média da perda por mandato, organizando por partidos

In [162]:
mediapercentpart = a/b
s = mediapercentpart.sort_values(ascending=True)
mptd = pd.DataFrame({'sigla_partido':s.index, 'média_por_mandato': s.values})
mptd

,sigla_partido,média_por_mandato
0,PSB,0.042159
1,PSL,0.142464
2,PMN,0.212082
3,PROS,0.292676
4,PTB,0.453156
5,PPB,0.822836
6,PP,0.902800
7,PDT,1.096243
8,PT,1.156398
9,PSDB,1.282231


<h3> Queremos mostrar um gráfico de barras com as informações de quais são os partidos que mais permitem a perda da biodiversidade

In [163]:
fig = px.bar(mptd, y='média_por_mandato', x='sigla_partido' ,title='Porcentagem de perda do bioma da amazônia por mandato')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()

<h3>Vamos selecionar os estados que tiveram perdas mais massivas, comparativamente, em relaçao ao bioma da amazônia

<p>Queremos a perda total no espaço amostral de tempo por cada estado, vamos selecionar o que teve mais área perdida comparativamente para visualizar os dados de como foram a sequencia de mandatos, e em qual mandato mais teve perda
   

In [172]:
perdaestado= deltadb.groupby('sigla_uf').delta.sum()

<p>Temos a informação da soma das perdas, mas ainda precisamos normalizar o dado, já que a cobertura de tempo de diferentes estados também é diferente

In [173]:
countestado=deltadb.groupby('sigla_uf').sigla_uf.count()

In [174]:
pestcpt= perdaestado/countestado
pestcpt.sort_values(ascending=False)

sigla_uf
MA    5.146821
MT    4.209027
RO    4.144400
TO    2.966543
PA    1.764645
AC    1.174981
RR    0.500949
AM    0.185513
AP    0.064529
dtype: float64

<b> Vemos que comparativamente, as unidades federais que mais sofreram foram: Maranhão, Mato Grosso e Rondônia

<h3>Modelando os dados dos 3 estados que mais sofreram para poder plotar em gráficos:

<p> Abaixo, modelamos um novo data frame que esteja organizado por estado e ao longo dos anos tenha sua respectiva áreatotal.
    Modelamos desse jeito para poder plotar em um gráfico de linhas.

In [127]:
col = pd.unique(deltadb["sigla_uf"])
cobvis=csvCob.query('ano >= 1994 and ano <= 2018 and (estado_abrev == "MA" or estado_abrev == "MT" or estado_abrev == "RO" )')
cobmavis = cobvis.query('estado_abrev == "MA"')
cobmtvis = cobvis.query('estado_abrev == "MT"')
cobrovis = cobvis.query('estado_abrev == "RO"')
newdf = pd.DataFrame(columns = col)

newdf["ano"]=pd.unique(cobvis["ano"])


newdf = newdf.merge(cobmavis,how = 'inner', on = 'ano').merge(cobmtvis,how= 'inner', on = 'ano').merge(cobrovis,how= 'inner', on = 'ano')
newdf = newdf[['ano','areatotal_x','areatotal_y', 'areatotal']]
newdf = newdf.rename(columns = {'areatotal_x': 'MA', 'areatotal_y': 'MT', 'areatotal' : 'RO'})
newdf.head()

,ano,MA,MT,RO
0,1994,75481.156639,433299.150186,195376.592716
1,1995,74272.545092,429766.345561,191833.580550
2,1996,72223.250740,422290.605462,188182.214708
3,1997,72837.223173,412144.718601,186252.888139
4,1998,71104.771291,409861.953011,183489.029383


<b>Aqui usamos a biblioteca plotly para mostrar os dados:

In [126]:
fig = px.line(newdf, x='ano', y=newdf.columns,
             title='Área total(km²) do bioma amazônia nesses estados entre 1994 e 2018',
            line_shape = 'spline')

fig.show()

<h3> Vamos explorar os dados e procurar quais foram os governos desses estados para ver qual foi mais nocivo para o bioma da amazônia

<p> Selecionando dados referentes aos 3 estados que mais sofreram com a perda do bioma amazônia e adicionando os mandatos:

<b> Abaixo adicionamos o governo em dado mandato

In [201]:
rodb= deltadb.query('sigla_uf == "RO" or sigla_uf == "MT" or sigla_uf == "MA"')
rodb["governo_ano"]=  "("+ rodb["sigla_partido"] + ")"+rodb["ano"].astype(str) + "-" + ((rodb["ano"] + 4 ).astype(str)) 
rodb = rodb.rename(columns = {'perdatot' : 'perda_total', 'governo_ano' : 'governo_quadrienio'})
rodb

,ano,sigla_uf,areatotal,sigla_partido,delta,perda_total,governo_quadrienio
21,1994,MA,75481.156639,PFL,5.797984,4376.385349,(PFL)1994-1998
22,1998,MA,71104.771291,PFL,8.05166,5725.114204,(PFL)1998-2002
23,2002,MA,65379.657086,PFL,7.14347,4670.376412,(PFL)2002-2006
24,2006,MA,60709.280675,PDT,4.124277,2503.819022,(PDT)2006-2010
25,2010,MA,58205.461653,PMDB,3.161876,1840.384696,(PMDB)2010-2014
26,2014,MA,56365.076957,PC do B,2.601661,1466.428162,(PC do B)2014-2018
28,1998,MT,409861.953011,PSDB,6.03587,24738.735802,(PSDB)1998-2002
29,2002,MT,385123.217209,PPS,8.91838,34346.750475,(PPS)2002-2006
30,2006,MT,350776.466734,PPS,2.140185,7507.266226,(PPS)2006-2010
31,2010,MT,343269.200508,PMDB,-0.258326,-886.752105,(PMDB)2010-2014


<b> Abaixo queremos ver as informações do estado do Mato Grosso

In [187]:
mtdb= rodb.query('sigla_uf == "MT"')
mtdb = mtdb[['governo_quadrienio','perda_total']]
mtdb

,governo_quadrienio,perda_total
28,(PSDB)1998-2002,24738.735802
29,(PPS)2002-2006,34346.750475
30,(PPS)2006-2010,7507.266226
31,(PMDB)2010-2014,-886.752105


<b> Gráfico:

In [194]:
fig = px.bar(mtdb, x="governo_quadrienio", y="perda_total", title="Perda de cobertura do bioma em km² por mandato no Mato Grosso")
fig.show()

<p> Concluímos que no estado do Mato Grosso, na cobertura de tempo de 1998 a 2014, o governo que mais permitiu a perda do bioma foi o do partido PPS.

<b> Abaixo queremos ver as informações do estado do Maranhão

In [202]:
madb= rodb.query('sigla_uf == "MA"')
madb = madb[['governo_quadrienio','perda_total']]
madb

,governo_quadrienio,perda_total
21,(PFL)1994-1998,4376.385349
22,(PFL)1998-2002,5725.114204
23,(PFL)2002-2006,4670.376412
24,(PDT)2006-2010,2503.819022
25,(PMDB)2010-2014,1840.384696
26,(PC do B)2014-2018,1466.428162


<b> Gráfico:

In [203]:
fig = px.bar(madb, x="governo_quadrienio", y="perda_total", title="Perda de cobertura do bioma em km² por mandato no Maranhão")
fig.show()

<p> Concluímos que no estado do Maranhão, na cobertura de tempo de 1994 a 2018, o governo que mais permitiu a perda do bioma foi o do partido PFL.

<b> Abaixo queremos ver as informações do estado de Rondônia

In [206]:
rondb= rodb.query('sigla_uf == "RO"')
rondb = rondb[['governo_quadrienio','perda_total']]
rondb

,governo_quadrienio,perda_total
39,(PFL)1998-2002,9293.785503
40,(PSDB)2002-2006,13747.831667
41,(PPS)2006-2010,2659.415745
42,(PMDB)2010-2014,3187.870788
43,(PMDB)2014-2018,6318.378338


<b> Gráfico:

In [208]:
fig = px.bar(rondb, x="governo_quadrienio", y="perda_total", title="Perda de cobertura do bioma em km² por mandato em Rondônia")
fig.show()

<p> Concluímos que no estado de Rondônia, na cobertura de tempo de 1998 a 2018, o governo que mais permitiu a perda do bioma foi o do partido PSDB.